                                                                                        Patricia Grau Francitorra
# Exploratory article analysis
## Objective
1. Get data from Wikipedia - analyse html format
2. UD tagging
3. Treebanks

In [45]:
import re
import requests
from bs4 import BeautifulSoup # first pip install bs4
# format is shift alt f (in VSC)

### 1. Get data from Wikipedia

In [86]:
# Neat idea, but doesn't work with disambiguating urls
list_of_langs = ['en','sv' , 'ca', 'es', 'fr']
list_of_urls = [f'https://{lang}.wikipedia.org/wiki/Jupiter' for lang in list_of_langs]
list_of_urls

['https://en.wikipedia.org/wiki/Jupiter',
 'https://sv.wikipedia.org/wiki/Jupiter',
 'https://ca.wikipedia.org/wiki/Jupiter',
 'https://es.wikipedia.org/wiki/Jupiter',
 'https://fr.wikipedia.org/wiki/Jupiter']

In [87]:
eng_jup_url = 'https://en.wikipedia.org/wiki/Jupiter'
sv_jup_url = 'https://sv.wikipedia.org/wiki/Jupiter'
fr_jup_url = 'https://fr.wikipedia.org/wiki/Jupiter_(planète)' # disambiguation issue
spa_jup_url = 'https://es.wikipedia.org/wiki/Júpiter_(planeta)' # disambiguation issue
ca_jup_url = 'https://ca.wikipedia.org/wiki/Júpiter_(planeta)' # disambiguation issue

In [88]:
with open('jupiter_en.html', 'w') as f:
    f.write(requests.get(url = 'https://en.wikipedia.org/wiki/Jupiter').text)
with open('jupiter_spa.html', 'w') as g:
    g.write(requests.get(url = spa_jup_url).text)

In [94]:
def cleantext(file):
    f = open(file)
    # print(f.readlines())
    for alltext in f.readlines():
        texto = alltext.strip('\t')
        if texto.startswith('<p>') or texto.startswith('</p><p>'): # if '<p>' in texto or '</p>' in texto:
            cleantext = BeautifulSoup(texto, "html.parser").text # when using "lxml" as the parser insead of html.parser, some paragraphs/lines are missing
            cleantext = re.sub(r'\[\d+\]', '', cleantext) # remove references [digit]
            cleantext = cleantext.strip()
            if len(cleantext) != 0:
                print(cleantext)
                print()
    f.close()
    # TO DO - save to a file or use it for UD tagging

In [95]:
cleantext('jupiter_en.html')

Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass more than two and a half times that of all the other planets in the Solar System combined, but slightly less than one-thousandth the mass of the Sun. Jupiter is the third brightest natural  object in the Earth's night sky after the Moon and Venus. People have been observing it since prehistoric times; it was named after the Roman god Jupiter, the king of the gods, because of its observed size.

Jupiter is primarily composed of hydrogen, but helium constitutes one-quarter of its mass and one-tenth of its volume. It likely has a rocky core of heavier elements, but, like the other giant planets, Jupiter lacks a well-defined solid surface. The ongoing contraction of its interior generates heat greater than the amount received from the Sun. Because of its rapid rotation, the planet's shape is an oblate spheroid; it has a slight but noticeable bulge around the equator. The outer atmosp

In [96]:
cleantext('jupiter_spa.html')

Júpiter es el planeta más grande del sistema solar y el quinto en orden de lejanía al Sol.​ Es un gigante gaseoso que forma parte de los denominados planetas exteriores. Recibe su nombre del dios romano Júpiter (Zeus en la mitología griega). Es uno de los objetos naturales más brillantes en un cielo nocturno despejado, superado solo por la Luna, Venus y algunas veces Marte.​

Se trata del planeta que ofrece un mayor brillo a lo largo del año dependiendo de su fase. Es, además, después del Sol, el mayor cuerpo celeste del sistema solar, con una masa casi dos veces y media la de los demás planetas juntos (con una masa 318 veces mayor que la de la Tierra y tres veces mayor que la de Saturno, además de ser, en cuanto a volumen, 1321 veces más grande que la Tierra). También es el planeta más antiguo del sistema solar, siendo incluso más antiguo que el Sol; este descubrimiento fue realizado por investigadores de la universidad de Münster en Alemania.​​

Júpiter es un cuerpo masivo gaseoso, f

### 2. UD tagging